In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions

from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#load pcd file, filter, downsample
pcd = o3d.io.read_point_cloud("final_cropped.pcd")
#pcd_flat = o3d.io.read_point_cloud("pcd_flat.pcd")
#pcd_flat.paint_uniform_color([0,1,0])
pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd_flat])


In [67]:
ground_plane = np.asarray([0,0,1,0])
pcd_ground_plane, inliers = functions.detect_ground_plane(pcd)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

#trans = compute_plane_transformation(ground_plane, pcd_ground_plane)
trans = functions.calculate_floor_alignment_matrix(pcd_ground_plane)
print(trans)
test = cp.deepcopy(pcd)
pcd_t = test.transform(trans)

o3d.visualization.draw_geometries([pcd_t, pcd_flat])

[[ 9.99558991e-01  7.41309839e-04  2.96862533e-02 -1.49097932e-02]
 [ 0.00000000e+00  9.99688358e-01 -2.49637031e-02  1.25284777e-02]
 [-2.96955077e-02  2.49526939e-02  9.99247487e-01  5.01711355e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [68]:
#o3d.io.write_point_cloud("corridor_ground_align.pcd", pcd_t)

In [69]:
bb = pcd_t.get_axis_aligned_bounding_box()
min_bound = bb.get_min_bound()
max_bound = bb.get_max_bound()

min_bound[0] = -47
min_bound[1] = -3
min_bound[2] = - 2

max_bound[0] = 3
#max_bound[1] =
max_bound[2] = 3
print(min_bound, max_bound)

pcd_new = cp.deepcopy(pcd_t)
crob_bb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_new = pcd_new.crop(crob_bb)


o3d.visualization.draw_geometries([pcd_new, pcd_flat])

[-47.  -3.  -2.] [ 3.         10.16708183  3.        ]


In [70]:
o3d.io.write_point_cloud("final_cropped_ground_align2.pcd", pcd_t)

True